In [11]:
import os
import json
from pathlib import Path

import openai

from summarize import chunk_a_text

In [13]:
openai.api_key = os.environ['OPENAI']

# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [19]:
FIRST_TOKEN = {"role": "system", "content": "You are a helpful assistant that helps a user understand a video."}
def summarize_w_turbo_1step(text):
    resp = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            FIRST_TOKEN,
            {"role": "user", "content": f"{text}\n\nWrite a summary for the text above"}
      ]
    )
    return resp['choices'][0]['message']['content']

In [34]:
def summarize_w_turbo_2steps(text):
    resp = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            FIRST_TOKEN,
            {"role": "user", "content": f"{text}\n\nWrite main ideas as bullet points"}
      ]
    )
    ideas = resp['choices'][0]['message']['content']
    
    return summarize_w_turbo_1step(ideas)


def summarize_w_turbo_2steps_additive(text):
    r1 = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            FIRST_TOKEN,
            {"role": "user", "content": f"{text}\n\nWrite main ideas as bullet points"}
      ]
    )
    ideas = r1['choices'][0]['message']['content']
    
    r2 = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            FIRST_TOKEN,
            {"role": "user", "content": f"{text}\n\nWrite main ideas as bullet points"},
            {"role": "assistant", "content": ideas},
            {"role": "user", "content": "Write a summary for the text above given main ideas"}
      ]
    )
    return r2['choices'][0]['message']['content']

In [25]:
metafn = Path("/mnt/storage/newsboy/SUReQwwTq1M-meta.json")
with metafn.open() as f:
    m = json.load(f)
text = ' '.join(x['text'] for x in m['captions'])

In [26]:
chunks = list(chunk_a_text(text, max_tokens=2000))

In [27]:
chunks[0]

"hello everyone and welcome to commodity  culture where we break down the  commodity space for both new and  experienced investors before we get  started standard disclaimer nothing here  is investing advice do your own due  diligence and today we are once again  joined by the founder of Peter grandage  and Company and one of the most  respected commentators in the commodity  Space Mr Peter grandich it's great to  have you back on the show good to be  back with you now I want to touch on two  important points you made on your blog  recently that I think it's very  important for investors to understand  you said one failure is the norm in the  junior resource business doesn't mean  there's any dishonesty just means  despite best intentions not everyone  finds a future mine and number two  perhaps more importantly you have to be  both financially and mentally prepared  to lose part or all of your Capital so  how does one financially and mentally  prepare to lose part or all of their  Cap

In [36]:
summarize_w_turbo_1step(chunks[0])

"The founder of Peter Grandich and Company, Mr. Peter Grandich, joins a Commodity Culture podcast to discuss the commodity space. He highlights the risk of investing in the junior resource business and encourages investors to mentally and financially prepare for the possibility of losing some or all of their capital. He also points out the bullishness of gold due to central banks' significant buys and notes that gold is an underinvested asset in the US. He comments on the push towards a green economy and the minerals like copper and lithium needed to achieve it, saying that it will take time to reach the predicted level, but the push is on."

In [29]:
summarize_w_turbo_2steps(chunks[0])

'In the video, the importance of understanding the risks in the junior resource business both financially and mentally was emphasized. The central banks are buying gold as a diversification strategy and as a hedge against a potential decline of the US dollar. However, it may be difficult to fight against the global trend towards green energy and government regulations. While the demand for battery metals such as copper, lithium, and graphite are increasing with the push for electric vehicles, the infrastructure needs for the new green economy are not yet in place. It is essential to note that nothing in the video should be construed as investment advice.'

In [35]:
summarize_w_turbo_2steps_additive(chunks[0])

'In this video, Peter Grandich, the founder of Peter Grandich and Company, discusses various aspects of investing in commodities. He advises investors to be prepared financially and mentally to lose their capital and talks about how the junior resource market has a high failure rate. He also emphasizes the need for diversification into physical gold and talks about how gold is an under-invested asset in the US. Additionally, he comments on the worldwide push for a green economy and the need for the infrastructure to catch up with the technology to supply the demand for electric cars, and other green technologies. Lastly, he stated that battery metals would be in demand, but talks about the ups and downs of the commodity market.'